In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.models import resnet18
from skimage.feature import hog
import numpy as np

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        bottleneck_channels = out_channels // 4

        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, stride=stride)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)
        self.conv2 = nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)
        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ROIsProposal(nn.Module):
    def __init__(self, use_stn=True):
        super(ROIsProposal, self).__init__()
        self.use_stn = use_stn
        if use_stn:
            self.stn1 = self._make_stn()
            self.stn2 = self._make_stn()

    def _make_stn(self):
        # Using ResNet-18 as the localization network
        model = resnet18(weights="IMAGENET1K_V1")
        model.fc = nn.Linear(model.fc.in_features, 6)
        return model

    def forward(self, x):
        # Assume x is a batch of images resized to (512, 512)
        crops = []
        if self.use_stn:
            theta1 = self.stn1(x)
            grid1 = F.affine_grid(theta1.view(-1, 2, 3), size=(x.size(0), 3, 224, 224), align_corners=False)
            crops.append(F.grid_sample(x, grid1, align_corners=False))

            theta2 = self.stn2(x)
            grid2 = F.affine_grid(theta2.view(-1, 2, 3), size=(x.size(0), 3, 224, 224), align_corners=False)
            crops.append(F.grid_sample(x, grid2, align_corners=False))

        else:
            # Random cropping logic
            for _ in range(2):
                i, j, h, w = T.RandomCrop.get_params(x, output_size=(224, 224))
                crop = T.functional.crop(x, i, j, h, w)
                crops.append(crop)

        # Low-resolution crop
        low_res = F.interpolate(x, size=(256, 256), mode='bilinear', align_corners=False)
        i, j, h, w = T.RandomCrop.get_params(low_res, output_size=(224, 224))
        crops.append(T.functional.crop(low_res, i, j, h, w))

        return crops

class HandCraftedFeatures(nn.Module):
    def __init__(self):
        super(HandCraftedFeatures, self).__init__()

    def forward(self, x):
        # Convert tensor to numpy for feature extraction
        x_np = x.cpu().numpy().transpose(0, 2, 3, 1)  # N, H, W, C
        features = []
        for img in x_np:
            gray = 0.299 * img[:, :, 0] + 0.587 * img[:, :, 1] + 0.114 * img[:, :, 2]
            hog_features = hog(gray, orientations=6, pixels_per_cell=(16, 16),
                               cells_per_block=(2, 2), block_norm='L2-Hys')
            features.append(hog_features)
        features = np.array(features)
        return torch.tensor(features, dtype=torch.float32, device=x.device)

class MultibranchNetwork(nn.Module):
    def __init__(self, out_classes, use_stn=True):
        super(MultibranchNetwork, self).__init__()

        # ROI proposal module
        self.roi_proposal = ROIsProposal(use_stn=use_stn)

        # Shared layers for three branches
        self.branch1 = self._make_branch()
        self.branch2 = self._make_branch()
        self.branch3 = self._make_branch()

        # Hand-crafted feature injection
        self.hand_crafted_features = HandCraftedFeatures()

        # Concatenation processing
        self.join_layer = ResidualBlock(768, 256, stride=1)

        # Common deep layers
        self.resblock1 = ResidualBlock(256, 512, stride=2)
        self.resblock2 = self._make_residual_block(512, 1024, num_blocks=5, stride=2)
        self.resblock3 = self._make_residual_block(1024, 2048, num_blocks=3, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Task-specific fully connected layers
        #self.fc_artist = nn.Linear(2048 + 4056, 1508)  # HOG feature size is 3780
        # self.fc_style = nn.Linear(2048 + 4056, 125)
        # self.fc_genre = nn.Linear(2048 + 4056, 41)
        #self.fc_artist = nn.Linear(2048 + 4056, 1508)  # HOG feature size is 3780
        self.fc_artist = nn.Linear(2048 + 4056, out_classes)

    def _make_branch(self):
        layers = [
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            self._make_residual_block(64, 256, num_blocks=3, stride=1)
        ]
        return nn.Sequential(*layers)

    def _make_residual_block(self, in_channels, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(ResidualBlock(in_channels, out_channels, stride=s))
            in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        # ROI proposal
        crops = self.roi_proposal(x)
        x1, x2, x3 = crops

        # Process each branch
        out1 = self.branch1(x1)
        out2 = self.branch2(x2)
        out3 = self.branch3(x3)

        # Concatenate feature maps along the channel dimension
        out = torch.cat([out1, out2, out3], dim=1)
        out = self.join_layer(out)

        # Common processing
        out = self.resblock1(out)
        out = self.resblock2(out)
        out = self.resblock3(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)

        # Hand-crafted features
        crop_size = 224
        x_cropped = T.CenterCrop(crop_size)(x)
        hand_crafted = self.hand_crafted_features(x_cropped)

        #print(x.shape, out.shape, hand_crafted.shape)
        out = torch.cat([out, hand_crafted], dim=1)

        #print(out.shape)

        # Task-specific outputs
        artist = self.fc_artist(out)

        return artist

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = MultibranchNetwork(out_classes = 161, use_stn=True)

In [3]:
count_params = lambda pl: sum(p.numel() for p in pl)

In [4]:
count_params(model.parameters())

45670021

In [5]:
!pip install pytorch_memlab

In [7]:
model = model.to(DEVICE)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
BATCH_SIZE = 32
placeholder = torch.tensor([42] * BATCH_SIZE).to(DEVICE)

# Input batch of images
for _ in range(10):
    for _ in range(10):
        optimizer.zero_grad()
        inputs = torch.randn(BATCH_SIZE, 3, 512, 512).to(DEVICE)
        labels = torch.randint(0, 160, (BATCH_SIZE,)).to(DEVICE)

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


In [9]:
torch.cuda.empty_cache()

In [11]:
from pytorch_memlab import MemReporter
x = torch.randn(32, 3, 512, 512).to(DEVICE)
with torch.no_grad():
    r = model(x)

m = MemReporter()
m.report()

/usr/local/lib/python3.10/dist-packages/pytorch_memlab/mem_reporter.py:65: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  tensors = [obj for obj in objects if isinstance(obj, torch.Tensor)]
/usr/local/lib/python3.10/dist-packages/pytorch_memlab/mem_reporter.py:95: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  fact_numel = tensor.storage().size()


Element type                                            Size  Used MEM
-------------------------------------------------------------------------------
Storage on cuda:0
Tensor0                                    (32, 3, 512, 512)    96.00M
Tensor1                                            (32, 161)    20.50K
Parameter2                                     (64, 3, 7, 7)    37.00K
Parameter2.grad                                (64, 3, 7, 7)    37.00K
Parameter3                                             (64,)   512.00B
Parameter3.grad                                        (64,)   512.00B
Parameter4                                             (64,)   512.00B
Parameter4.grad                                        (64,)   512.00B
Parameter5                                             (64,)   512.00B
Parameter5.grad                                        (64,)   512.00B
Parameter6                                    (64, 64, 1, 1)    16.00K
Parameter6.grad                               (64,